# **Nhận dạng ngôn ngữ ký hiệu Việt Nam (bản chữ cái)**
*Hoàng Anh Hùng*

Notebook này tiến hành huấn luyện model LSTM nhận dạng ngôn ngữ ký hiệu thời gian thực.
- **Dữ liệu**: 33 ký hiệu (29 ký hiệu tiếng việt và 4 ký hiệu tiếng anh), mỗi ký hiệu có 30 videos thể hiện ký hiệu đó. Mỗi video trích xuất ra được 3 file (từ video gốc và 2 videos tăng cường) đặc trưng chứa thông tin vị trí tương đối của các điểm mốc trên bàn tay phải và khoảng cách thay đổi của bàn tay qua từng frame.
- **Đầu ra**:
  - Model: `model/best_model.h5`
  - Các biểu đồ, thang điểm đánh giá kết quả nhận dạng.


In [28]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import seaborn as sns

print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

BASE_DIR = '/content/drive/MyDrive/Colab Notebooks/SLR_alphabet'
DATA_DIR = "/content/alphabet_features"
MODEL_DIR = os.path.join(BASE_DIR, 'model')
EVALUATION_DIR = os.path.join(BASE_DIR, 'evaluation')
METADATA_PATH = "/content/alphabet_features/metadata.csv"

# tạo thư mục
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(EVALUATION_DIR, exist_ok=True)

TensorFlow version: 2.18.0
GPU available: []


## (Tùy chọn) Kết nối với Google Colab
Quá trình huấn luyện có thể thực hiện trên thiết bị cá nhân và chạy bằng CPU/GPU. Tuy nhiên có thể cải thiện tốc độ huấn luyện mô hình bằng cách tiến hành chạy trên Colab để tận dụng GPU ở đây nhằm nếu trên thiết bị không hỗ trợ GPU có kiến trúc CUDA.

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import pandas as pd
import os

zip_path = os.path.join(BASE_DIR, "alphabet_features.zip")

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(BASE_DIR)

In [11]:
!cp -r /content/drive/MyDrive/Colab\ Notebooks/SLR_alphabet/alphabet_features /content/

In [19]:
metadata_path = "/content/alphabet_features/metadata.csv"
df = pd.read_csv(metadata_path)

feature_base_path = os.path.join("/content", "alphabet_features")
df['feature_path'] = df['feature_path'].apply(
    lambda x: os.path.join(feature_base_path, os.path.basename(x.replace('\\', '/')))
)

df.to_csv(metadata_path, index=False)

In [25]:
with pd.option_context('display.max_colwidth', None):
  print(df.head(5))

   video_id label         type  \
0         1     A     original   
1         1     A   rotated_15   
2         1     A  rotated_-15   
3         2     A     original   
4         2     A   rotated_15   

                                         feature_path  
0     /content/alphabet_features/video_1_original.npy  
1   /content/alphabet_features/video_1_rotated_15.npy  
2  /content/alphabet_features/video_1_rotated_-15.npy  
3     /content/alphabet_features/video_2_original.npy  
4   /content/alphabet_features/video_2_rotated_15.npy  


## Huấn luyện mô hình SLR

In [30]:
def load_data(samples_per_label=30):
    #Tải dữ liệu
    if not os.path.exists(METADATA_PATH):
        raise FileNotFoundError(f"Metadata file not found: {METADATA_PATH}")

    metadata = pd.read_csv(METADATA_PATH)
    X, y = [], []
    count = 0

    labels = metadata['label'].unique()
    print(f"Found {len(labels)} labels: {labels}")

    for label in labels:
        label_data = metadata[metadata['label'] == label]
        video_ids = label_data['video_id'].unique()
        selected_vids = np.random.choice(video_ids, size=min(samples_per_label, len(video_ids)), replace=False)

        for vid in selected_vids:
            for vid_type in ['original', 'rotated_15', 'rotated_-15']:
                vid_data = label_data[(label_data['video_id'] == vid) & (label_data['type'] == vid_type)]
                if vid_data.empty:
                    print(f"Missing {vid_type} for video_id {vid}, label {label}")
                    continue
                feature_path = vid_data['feature_path'].iloc[0]
                features = np.load(feature_path)
                if features.shape == (30, 64):  # Check for 132-dimensional features
                    X.append(features)
                    y.append(label)
                    count += 1
                    if count % 100 == 0:
                        print(f"Loaded {count} samples")
                else:
                    print(f"Invalid shape {features.shape} for {feature_path}")

    if not X:
        raise ValueError("No valid data loaded")

    X = np.array(X)  # Shape: (n_samples, 30, 64)
    y = np.array(y)  # Shape: (n_samples,)

    # Encode labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_onehot = tf.keras.utils.to_categorical(y_encoded)  # Shape: (n_samples, n_classes)

    print(f"Loaded {len(X)} samples with {len(label_encoder.classes_)} classes")
    return X, y_onehot, label_encoder

In [4]:
def build_lstm_model(input_shape, num_classes):
    """Build LSTM model for sign language word recognition."""
    model = Sequential([
        LSTM(128, input_shape=input_shape, return_sequences=True),
        Dropout(0.3),
        LSTM(64),
        Dropout(0.3),
        BatchNormalization(),
        Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
def plot_training_history(history):
    """Plot training and validation loss/accuracy."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.plot(history.history['loss'], label='Train Loss')
    ax1.plot(history.history['val_loss'], label='Validation Loss')
    ax1.set_title('Model Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='Train Accuracy')
    ax2.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax2.set_title('Model Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()

    plt.tight_layout()
    plt.savefig(os.path.join(EVALUATION_DIR, 'training_history.png'))
    plt.close()

In [6]:
def plot_confusion_matrix(y_true, y_pred, label_encoder):
    """Plot confusion matrix."""
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
    plt.title('Confusion Matrix - LSTM')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(os.path.join(EVALUATION_DIR, 'confusion_matrix.png'))
    plt.close()

In [7]:
def train_and_evaluate():
    """Train and evaluate model."""
    # Load data
    try:
        X, y, label_encoder = load_data(samples_per_label=30)
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print(f"Train: {len(X_train)}, Test: {len(X_test)}")

    # Build model
    model = build_lstm_model(input_shape=(30, 64), num_classes=len(label_encoder.classes_))
    model.summary()

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint = ModelCheckpoint(
        os.path.join(MODEL_DIR, 'best_model.h5'), save_best_only=True, monitor='val_loss'
    )

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=50,
        batch_size=32,
        callbacks=[early_stopping, checkpoint],
        verbose=1
    )

    # Evaluate
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

    # Predict
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)

    # Save evaluation results
    with open(os.path.join(EVALUATION_DIR, 'evaluation_metrics.txt'), 'w') as f:
        f.write(f"Test Loss: {test_loss:.4f}\n")
        f.write(f"Test Accuracy: {test_accuracy:.4f}\n\n")
        f.write("Classification Report:\n")
        f.write(
            classification_report(
                y_true_classes,
                y_pred_classes,
                target_names=label_encoder.classes_
            )
        )

    # Plot
    plot_training_history(history)
    plot_confusion_matrix(y_true_classes, y_pred_classes, label_encoder)

In [33]:
train_and_evaluate()

Found 33 labels: ['A' 'AA' 'AW' 'B' 'C' 'D' 'DD' 'E' 'EE' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M'
 'N' 'O' 'OO' 'OW' 'P' 'Q' 'R' 'S' 'T' 'U' 'UW' 'V' 'W' 'X' 'Y' 'Z']
Loaded 100 samples
Loaded 200 samples
Loaded 300 samples
Loaded 400 samples
Loaded 500 samples
Loaded 600 samples
Loaded 700 samples
Loaded 800 samples
Loaded 900 samples
Loaded 1000 samples
Loaded 1100 samples
Loaded 1200 samples
Loaded 1300 samples
Loaded 1400 samples
Loaded 1500 samples
Loaded 1600 samples
Loaded 1700 samples
Loaded 1800 samples
Loaded 1900 samples
Loaded 2000 samples
Loaded 2100 samples
Loaded 2200 samples
Loaded 2300 samples
Loaded 2400 samples
Loaded 2500 samples
Loaded 2600 samples
Loaded 2700 samples
Loaded 2800 samples
Loaded 2900 samples
Loaded 2970 samples with 33 classes
Train: 2376, Test: 594


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 33)             │         2,145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 154,785 (604.63 KB)

...

75/75 ━━━━━━━━━━━━━━━━━━━━ 11s 81ms/step - accuracy: 0.9981 - loss: 0.0871 - val_accuracy: 0.9966 - val_loss: 0.0754
Epoch 40/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 105ms/step - accuracy: 1.0000 - loss: 0.0836 - val_accuracy: 0.9949 - val_loss: 0.0798
Epoch 41/50
74/75 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9994 - loss: 0.0763

75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.9994 - loss: 0.0764 - val_accuracy: 0.9949 - val_loss: 0.0724
Epoch 42/50
74/75 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.9973 - loss: 0.0770

75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 74ms/step - accuracy: 0.9973 - loss: 0.0769 - val_accuracy: 0.9966 - val_loss: 0.0683
Epoch 43/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - accuracy: 0.9991 - loss: 0.0700 - val_accuracy: 0.9949 - val_loss: 0.0698
Epoch 44/50
74/75 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.9998 - loss: 0.0637

75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 75ms/step - accuracy: 0.9998 - loss: 0.0638 - val_accuracy: 0.9966 - val_loss: 0.0630
Epoch 45/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 101ms/step - accuracy: 1.0000 - loss: 0.0628 - val_accuracy: 0.9966 - val_loss: 0.0631
Epoch 46/50
74/75 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9972 - loss: 0.0659

75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 75ms/step - accuracy: 0.9972 - loss: 0.0659 - val_accuracy: 0.9949 - val_loss: 0.0599
Epoch 47/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 101ms/step - accuracy: 0.9993 - loss: 0.0616 - val_accuracy: 0.9949 - val_loss: 0.0621
Epoch 48/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 8s 72ms/step - accuracy: 0.9995 - loss: 0.0605 - val_accuracy: 0.9949 - val_loss: 0.0626
Epoch 49/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 10s 67ms/step - accuracy: 0.9999 - loss: 0.0585 - val_accuracy: 0.9949 - val_loss: 0.0611
Epoch 50/50
75/75 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 1.0000 - loss: 0.0555 - val_accuracy: 0.9949 - val_loss: 0.0691

Test Loss: 0.0599, Test Accuracy: 0.9949
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step
